In [3]:
#사전 학습된 모델로 영화 리뷰 긍부정 판별기 

In [4]:
from datasets import load_dataset
data = load_dataset('rotten_tomatoes')

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [5]:
#pipeline를 이용한 모델과 토큰나이저 가져오
from transformers import pipeline 

model_id = "cardiffnlp/twitter-roberta-base-sentiment-latest"

pipe = pipeline(
    model = model_id,
    tokenizer=model_id,
    return_all_scores = True,
    device='cuda'
)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda


In [6]:
import numpy as np 
from tqdm import tqdm 
from transformers.pipelines.pt_utils import KeyDataset

y_pred = []
for output in tqdm(pipe(KeyDataset(data['test'], 'text'))):
    negative_score = output[0]['score']
    positive_score = output[2]['score']
    assigment = np.argmax([negative_score, positive_score ])
    y_pred.append(assigment)

100%|██████████| 1066/1066 [00:09<00:00, 107.96it/s]


In [7]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    performance = classification_report(y_true, y_pred, 
                         target_names=['Negative Review', 'Positive Review'])
    print (performance)

evaluate_performance(data['test']['label'],y_pred )


                 precision    recall  f1-score   support

Negative Review       0.76      0.88      0.81       533
Positive Review       0.86      0.72      0.78       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066



In [8]:
#임베딩된 문자을 로지스틱을 이용해서 학습 
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

train_embeddings = model.encode(data['train']['text'])
test_embeddings  = model.encode(data['test']['text'])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/267 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

In [9]:
train_embeddings.shape

(8530, 768)

In [10]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(train_embeddings, data['train']['label'])

LogisticRegression()

In [12]:
y_pred = clf.predict(test_embeddings)

In [13]:
evaluate_performance(data['test']['label'],y_pred )

                 precision    recall  f1-score   support

Negative Review       0.85      0.86      0.85       533
Positive Review       0.86      0.85      0.85       533

       accuracy                           0.85      1066
      macro avg       0.85      0.85      0.85      1066
   weighted avg       0.85      0.85      0.85      1066



In [14]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer2 = AutoTokenizer.from_pretrained("nlp04/korean_sentiment_analysis_kcelectra")
model2 = AutoModelForSequenceClassification.from_pretrained("nlp04/korean_sentiment_analysis_kcelectra")
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe2 = pipeline("text-classification", model="nlp04/korean_sentiment_analysis_kcelectra")

pipe2("오늘은 짜장면 먹는 날이다")


tokenizer_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/511M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/511M [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': '즐거운(신나는)', 'score': 0.765084981918335}]

In [27]:
answer = pipe2("오늘은 짜장면 먹는 날이다")
answer[0]['label']

'즐거운(신나는)'

In [15]:
pip install discord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
